In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy.interpolate import make_interp_spline, UnivariateSpline
import numpy as np
import json
from json import JSONEncoder
import ast
import seaborn as sns
import warnings
os.environ['USE_PYGEOS']='0' # Hva er dette?

# Display all columns in the df
pd.set_option('display.max_columns', None)
# Set the float format to display numbers in standard decimal notation
pd.options.display.float_format = '{:.5f}'.format
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Original

In [2]:
# Define the kW types of chargers
kW_list = [350, 300, 250, 200, 180, 175, 150, 135, 129, 125, 120, 100, 75, 62.5, 62, 60, 55, 50, 48, 44, 43]

# Define the average battery capacity, Source: https://www.solarquotes.com.au/blog/norway-ev-batteries/
average_battery_capacity = 68

# Define the battery aging factor (80% to 90% efficiency due to aging)
battery_aging_ratio = 0.9

average_battery_capacity*=battery_aging_ratio

# Define the setup and takedown time (in minutes)
setup_takedown_time = 5

# Convert setup and takedown time to hours
setup_takedown_time_hours = setup_takedown_time / 60

# Initialize an empty dictionary
cars_per_hour_dict = {}

# Loop through the kW list
for kW in kW_list:
    # Calculate charging time
    charging_time = average_battery_capacity / kW

    # Add the setup and takedown time to the charging time
    total_time = charging_time + setup_takedown_time_hours

    # Calculate cars per hour and round it to 1 decimal place
    cars_per_hour = round(1 / total_time, 1)

    # Add this to the dictionary
    cars_per_hour_dict[kW] = cars_per_hour

# Print out the resulting dictionary
for kW, cars_per_hour in cars_per_hour_dict.items():
    print(f"{kW}: {cars_per_hour}")


350: 3.9
300: 3.5
250: 3.0
200: 2.6
180: 2.4
175: 2.3
150: 2.0
135: 1.9
129: 1.8
125: 1.7
120: 1.7
100: 1.4
75: 1.1
62.5: 0.9
62: 0.9
60: 0.9
55: 0.8
50: 0.8
48: 0.7
44: 0.7
43: 0.7


In [13]:

#Sources: https://elbilstatistikk.no / https://www.motor.no/elbil-elbilbestand-statistikk/fem-modeller-utgjor-over-halvparten-av-elbilene-i-norge/186847


data = {
    'CarType': ['Jaguar I-Pace', 'VW ID-3', 'Nissan e-NV200', 'Hyundai Kona', 'Volkswagen UP!', 'Hyundai Ioniq', 'Tesla Model X', 'Renault Zoe', 'Audi E-Tron', 'Tesla Model 3', 'Kia Soul', 'Tesla Model S', 'BMW i3', 'VW e-Golf', 'Nissan Leaf', 'Other'],
    'MaxChargeType': [100, 135, 46, 50, 50, 350, 250, 46, 170, 170, 77, 250, 49, 43, 46, 55],
    'PercentageOfCars': ['2 %', '2 %', '2 %', '3 %', '3 %', '3 %', '4 %', '4 %', '4 %', '6 %', '6 %', '6 %', '8 %', '14 %', '19 %', '15 %'],
    'Percentage': [1.6, 2.16, 0.874, 1.35, 1.6, 11.55, 10, 1.886, 7.31, 9.52, 4.543, 15.25, 3.969, 5.934, 8.648, 8.25]
}

df = pd.DataFrame(data)
print(df)

average=df["Percentage"].sum()
print("Average Charger Type of Cars: ", average)


           CarType  MaxChargeType PercentageOfCars  Percentage
0    Jaguar I-Pace            100              2 %     1.60000
1          VW ID-3            135              2 %     2.16000
2   Nissan e-NV200             46              2 %     0.87400
3     Hyundai Kona             50              3 %     1.35000
4   Volkswagen UP!             50              3 %     1.60000
5    Hyundai Ioniq            350              3 %    11.55000
6    Tesla Model X            250              4 %    10.00000
7      Renault Zoe             46              4 %     1.88600
8      Audi E-Tron            170              4 %     7.31000
9    Tesla Model 3            170              6 %     9.52000
10        Kia Soul             77              6 %     4.54300
11   Tesla Model S            250              6 %    15.25000
12          BMW i3             49              8 %     3.96900
13       VW e-Golf             43             14 %     5.93400
14     Nissan Leaf             46             19 %     

### Given that the average charging type is far closer to 100 output speed than to the nearest lower charge type, we will round the average up to 100

In [11]:
# Define the kW types of chargers
kW_list = [350, 300, 250, 200, 180, 175, 150, 135, 129, 125, 120, 100, 75, 62.5, 62, 60, 55, 50, 48, 44, 43]

# Define the average battery capacity, Source: https://www.solarquotes.com.au/blog/norway-ev-batteries/
average_battery_capacity = 68

# Define the battery aging factor (80% to 90% efficiency due to aging)
battery_aging_ratio = 0.9

average_battery_capacity*=battery_aging_ratio

# Define the setup and takedown time (in minutes)
setup_takedown_time = 5

# Convert setup and takedown time to hours
setup_takedown_time_hours = setup_takedown_time / 60

# Initialize an empty dictionary
cars_per_hour_dict = {}

# Loop through the kW list
for kW in kW_list:
    # Calculate charging time
    charging_time = average_battery_capacity / kW

    # Add the setup and takedown time to the charging time
    total_time = charging_time + setup_takedown_time_hours

    # Calculate cars per hour and round it to 1 decimal place
    cars_per_hour = round(1 / total_time, 1)

    # Add this to the dictionary
    cars_per_hour_dict[kW] = cars_per_hour

    if kW >=101:   
        cars_per_hour_dict[kW]=1.4

# Print out the resulting dictionary
for kW, cars_per_hour in cars_per_hour_dict.items():
    print(f"{kW}: {cars_per_hour}")


350: 1.4
300: 1.4
250: 1.4
200: 1.4
180: 1.4
175: 1.4
150: 1.4
135: 1.4
129: 1.4
125: 1.4
120: 1.4
100: 1.4
75: 1.1
62.5: 0.9
62: 0.9
60: 0.9
55: 0.8
50: 0.8
48: 0.7
44: 0.7
43: 0.7
